# Assignment 03
Generate Girvan-Newman (GN) networks varying:
- $z_{in}, z_{out}$ or the probabilities matrix
- communities number
- communities size/proportion

Discuss the properties of networks spectrum. Compare it with random networks without communities (ER)